In [0]:
from gensim.models.wrappers import FastText
import pandas as pd
from google.colab import drive

In [0]:
drive.mount('/gdrive')
%cd /gdrive
%cd My\ Drive/
%cd Colab\ Notebooks

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive
/gdrive/My Drive
/gdrive/My Drive/Colab Notebooks


In [0]:
movies_dataset = pd.read_json('IMDB_reviews.json', lines=True)
movies_dataset.head()

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
0,10 February 2006,tt0111161,ur1898687,True,"In its Oscar year, Shawshank Redemption (writt...",10,A classic piece of unforgettable film-making.
1,6 September 2000,tt0111161,ur0842118,True,The Shawshank Redemption is without a doubt on...,10,Simply amazing. The best film of the 90's.
2,3 August 2001,tt0111161,ur1285640,True,I believe that this film is the best story eve...,8,The best story ever told on film
3,1 September 2002,tt0111161,ur1003471,True,"**Yes, there are SPOILERS here**This film has ...",10,Busy dying or busy living?
4,20 May 2004,tt0111161,ur0226855,True,At the heart of this extraordinary movie is a ...,8,"Great story, wondrously told and acted"


In [0]:
review_text = movies_dataset[["is_spoiler","review_text"]]
review_text.head()

,is_spoiler,review_text
0,True,"In its Oscar year, Shawshank Redemption (writt..."
1,True,The Shawshank Redemption is without a doubt on...
2,True,I believe that this film is the best story eve...
3,True,"**Yes, there are SPOILERS here**This film has ..."
4,True,At the heart of this extraordinary movie is a ...


In [0]:
review_text["label"] = review_text["is_spoiler"].apply(lambda x: 1 if x==True else 0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
review_text.tail()

,is_spoiler,review_text,label
573908,False,"Go is wise, fast and pure entertainment. Assem...",0
573909,False,"Well, what shall I say. this one´s fun at any ...",0
573910,False,"Go is the best movie I have ever seen, and I'v...",0
573911,False,Call this 1999 teenage version of Pulp Fiction...,0
573912,False,Why was this movie made? No doubt to sucker in...,0


In [0]:
review_text.shape
  

(573913, 3)

In [0]:
count =0
spolier_reviews = review_text[review_text["is_spoiler"]==True]
non_spolier_reviews = review_text[review_text["is_spoiler"]==False]

balanced_reviews = spolier_reviews[:25000].append(non_spolier_reviews[:25000], ignore_index=True)
balanced_reviews.count

<bound method DataFrame.count of        is_spoiler                                        review_text  label
0            True  In its Oscar year, Shawshank Redemption (writt...      1
1            True  The Shawshank Redemption is without a doubt on...      1
2            True  I believe that this film is the best story eve...      1
3            True  **Yes, there are SPOILERS here**This film has ...      1
4            True  At the heart of this extraordinary movie is a ...      1
...           ...                                                ...    ...
49995       False  This is pretty much the oldest feature film th...      0
49996       False  I still cant believe how funny this movie is. ...      0
49997       False  I Am Not Really A Big Chaplin Fan But This Is ...      0
49998       False  This is what you can call real art because thi...      0
49999       False  When sound movies were being made around that ...      0

[50000 rows x 3 columns]>

In [0]:
balanced_reviews["label"] = balanced_reviews["is_spoiler"].apply(lambda x: 1 if x==True else 0)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(balanced_reviews["review_text"], balanced_reviews["label"],test_size=0.30, random_state=1)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(strip_accents='ascii', token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b', lowercase=True, stop_words='english')
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

In [0]:
import pandas as pd
word_freq_df = pd.DataFrame(X_train_cv.toarray(), columns=cv.get_feature_names())
top_words_df = pd.DataFrame(word_freq_df.sum()).sort_values(0, ascending=False)

In [0]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_cv, y_train)
predictions = naive_bayes.predict(X_test_cv)

In [0]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
print("Accuracy score: ", accuracy_score(y_test, predictions))
print("Precision score: ", precision_score(y_test, predictions))
print("Recall score: ", recall_score(y_test, predictions))
print("F1 score:  ", (2*precision_score(y_test, predictions) * recall_score(y_test, predictions)) / (precision_score(y_test, predictions)+ recall_score(y_test, predictions)))

Accuracy score:  0.7955333333333333
Precision score:  0.8717115946735953
Recall score:  0.702158273381295
F1 score:   0.7778019271172933
